In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [10]:
pip install sktime --user

Looking in indexes: http://ftp.daumkakao.com/pypi/simple



In [2]:
train = pd.read_csv('./data/fs_train.csv', encoding='utf-8')
test = pd.read_csv('./data/fs_test.csv', encoding='utf-8')
train

,building_number,temperature,windspeed,power_consumption,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity,hour,...,building_type_Data Center,building_type_Department Store and Outlet,building_type_Discount Mart,building_type_Hospital,building_type_Hotel and Resort,building_type_Knowledge Industry Center,building_type_Other Buildings,building_type_Public,building_type_Research Institute,building_type_University
0,1,18.6,0.9,1085.28,110634.00,39570.00,0.0,0.0,0.0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,18.0,1.1,1047.36,110634.00,39570.00,0.0,0.0,0.0,1,...,0,0,0,0,0,0,1,0,0,0
2,1,17.7,1.5,974.88,110634.00,39570.00,0.0,0.0,0.0,2,...,0,0,0,0,0,0,1,0,0,0
3,1,16.7,1.4,953.76,110634.00,39570.00,0.0,0.0,0.0,3,...,0,0,0,0,0,0,1,0,0,0
4,1,18.4,2.8,986.40,110634.00,39570.00,0.0,0.0,0.0,4,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,26.2,2.0,1232.64,57497.84,40035.23,0.0,0.0,0.0,19,...,0,0,0,0,1,0,0,0,0,0
203996,100,25.8,2.0,1103.28,57497.84,40035.23,0.0,0.0,0.0,20,...,0,0,0,0,1,0,0,0,0,0
203997,100,25.6,0.7,964.56,57497.84,40035.23,0.0,0.0,0.0,21,...,0,0,0,0,1,0,0,0,0,0
203998,100,25.4,1.7,773.28,57497.84,40035.23,0.0,0.0,0.0,22,...,0,0,0,0,1,0,0,0,0,0


In [6]:
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    result = np.mean(v) * 100
    return result

In [ ]:
# X = train.drop('power_consumption', axis=1)
# y = train['power_consumption']

# scaler = StandardScaler()
# s_X = scaler.fit_transform(np.array(X))
# # s_y = scaler.fit_transform(np.array(y).reshape(-1,1))

# X_train, X_test, y_train, y_test = train_test_split(s_X, y, random_state=42)
# cols = X.columns


# # print(X_train.shape)

# params = {
#     'n_estimators': [100, 300, 500, 1000],
#     'max_depth': [3, 5, 7],
#     'min_child_weight' : [1,3,5,7],
#     'subsample': [0.8, 0.9]
# }

# gscv_xgb = GridSearchCV(estimator=XGBRegressor(), param_grid=params, scoring='neg_mean_squared_error',
#                           cv=5, refit=True, n_jobs=-1, verbose=1)

# gscv_xgb.fit(X_train, y_train)

# y_pred = gscv_xgb_1.predict(X_test)
# print(f"Train SMAPE: {smape(y_test, y_pred_1)}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [5]:
X = train.drop('power_consumption', axis=1)
y = train['power_consumption']

scaler = StandardScaler()
s_X = scaler.fit_transform(np.array(X))
# s_y = scaler.fit_transform(np.array(y).reshape(-1,1))

X_train, X_test, y_train, y_test = train_test_split(s_X, y, random_state=42)
# cols = X.columns

xgbr = XGBRegressor(objective='reg:squarederror',n_estimators=300, max_depth=5, min_child_weight=3, subsample=0.8)

xgbr.fit(X_train, y_train)

y_pred = xgbr.predict(X_test)
print(f"Train SMAPE: {smape(y_test, y_pred)}")

Train SMAPE: 9.42977092717028


In [9]:
s_test = scaler.fit_transform(np.array(test))

gogo = xgbr.predict(s_test)

submission = pd.read_csv('./data/sample_submission.csv', encoding='utf-8')
submission['answer'] = gogo
submission.to_csv('./submission/submission_xgb_01.csv', encoding='utf-8',index=False)

# 첫번째 제출 SMAPE : 13.581785971

In [8]:
from sktime.forecasting.model_selection import temporal_train_test_split
# temporal_train_test_split 차이를 두고 해보기

X = train.drop('power_consumption', axis=1)
y = train['power_consumption']

scaler = StandardScaler()
s_X = scaler.fit_transform(np.array(X))
# s_y = scaler.fit_transform(np.array(y).reshape(-1,1))

y_train, y_test, X_train, X_test = temporal_train_test_split(y=y, X= s_X, test_size=168)
# cols = X.columns

xgbr = XGBRegressor(objective='reg:squarederror',n_estimators=300, max_depth=5, min_child_weight=3, subsample=0.8)

xgbr.fit(X_train, y_train)

y_pred = xgbr.predict(X_test)
print(f"Train SMAPE: {smape(y_test, y_pred)}")

Train SMAPE: 9.093978353063488


In [9]:
s_test = scaler.fit_transform(np.array(test))

gogo = xgbr.predict(s_test)

submission = pd.read_csv('./data/sample_submission.csv', encoding='utf-8')
submission['answer'] = gogo
submission.to_csv('./submission/submission_xgb_02.csv', encoding='utf-8',index=False)